In [1]:
import boto3
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'Fall_2021/In_Class_Assignments/framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Defining the input and target variables
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [3]:
## Transforming 0-1 scale 
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_test.columns)

In [4]:
X_train.head()

,male,age,currentSmoker,totChol,sysBP,BMI,heartRate,glucose
0,1.0,0.289474,1.0,0.367983,0.333333,0.278586,0.291667,0.153846
1,1.0,0.342105,1.0,0.201663,0.336449,0.287508,0.406250,0.176638
2,0.0,0.447368,1.0,0.351351,0.096573,0.219762,0.343750,0.094017
3,1.0,0.394737,1.0,0.467775,0.118380,0.402842,0.218750,0.105413
4,0.0,0.263158,1.0,0.168399,0.221184,0.243556,0.135417,0.074074


In [5]:
X_test.head()

,male,age,currentSmoker,totChol,sysBP,BMI,heartRate,glucose
0,0.0,0.181818,0.0,0.410774,0.283688,0.241567,0.255102,0.121469
1,1.0,0.030303,1.0,0.303030,0.085106,0.197173,0.204082,0.096045
2,1.0,0.757576,0.0,0.427609,0.252955,0.323165,0.377551,0.135593
3,1.0,0.515152,0.0,0.663300,0.243499,0.323165,0.193878,0.096045
4,0.0,0.000000,1.0,0.185185,0.000000,0.005704,0.346939,0.064972


In [7]:
## LASSO CV
lasso_cv = LassoCV(cv = 5).fit(X_train, Y_train)
lasso_cv.alpha_

## Building LASSO regression
lasso_md = Lasso(alpha = lasso_cv.alpha_).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.06866468,  0.26944326,  0.04108615,  0.        ,  0.32243806,
       -0.        ,  0.        ,  0.33615801])

In [9]:
X_train_lasso = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_lasso = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

## Logistic regression
logit_md = LogisticRegression().fit(X_train_lasso, Y_train)

## Predicting on test
pred1 = logit_md.predict_proba(X_test_lasso)[:, 1]
pred1 = np.where(pred1 < 0.1, 0, 1)

## Computing the recall
recall_score(Y_test, pred1)

0.8303571428571429

In [11]:
## Random forest classifier
RF = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

## Extracting importance
RF.feature_importances_

array([0.02280941, 0.15057135, 0.02130881, 0.16683375, 0.18382614,
       0.17212698, 0.12438411, 0.15813945])

In [13]:
X_train_RF = X_train[['age', 'totChol', 'sysBP', 'BMI', 'glucose']]
X_test_RF = X_test[['age', 'totChol', 'sysBP', 'BMI', 'glucose']]

## Building the random forest model 
RF = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_RF, Y_train)

## Predicting on test
pred2 = RF.predict_proba(X_test_RF)[:, 1]
pred2 = np.where(pred2 < 0.1, 0, 1)

## Computing the recall
recall_score(Y_test, pred2)

0.9285714285714286